# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

# Download Data

In [2]:
class YahooFinance:
    def __init__(self, symbol:str, start_date:str, end_date:str, interval:str='1d'):
        self.symbol = symbol
        self.start_date = start_date
        self.end_date = end_date
        self.interval = interval
        self.col_names = ["date", 'price', 'log_return']
    
    def _download_data(self, symbol:str|list[str], start_date:str, end_date:str, interval:str='1d') -> pd.DataFrame:
        prices = yf.download(symbol, start=start_date, end=end_date, interval=interval, auto_adjust=False)
        prices.index = prices.index.date
        return prices
    
    def _filter_cols(self, prices:pd.DataFrame) -> pd.DataFrame:
        filtered_prices = prices['Adj Close'].copy()
        filtered_prices.columns = ['Adj Close']
        return filtered_prices
    
    def _pct_change(self, filtered_prices:pd.DataFrame) -> pd.DataFrame:
        df = filtered_prices.copy()
        df['log_return'] = np.log(df['Adj Close']/ df['Adj Close'].shift(1))
        return df
    
    def _reset_index(self, filtered_prices:pd.DataFrame) -> pd.DataFrame:
        df = filtered_prices.reset_index(names='Date', drop=False).copy()
        df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%d/%m/%Y')
        df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y') #Convert from string to datetime
        return df
    
    def pipeline(self) -> pd.DataFrame:
        prices = self._download_data(self.symbol, self.start_date, self.end_date, self.interval)
        filtered_prices = self._filter_cols(prices)
        df = self._reset_index(filtered_prices)
        df = self._pct_change(df)
        df.columns = self.col_names
        df.set_index('date', drop=True, inplace=True)
        return df

In [4]:
params = {
    'symbol': 'SPY',
    'start_date': '2000-01-01',
    'end_date': '2025-12-31'
}
yahoo = YahooFinance(**params)
df = yahoo.pipeline()
df.head(5)

[*********************100%***********************]  1 of 1 completed


,price,log_return
date,,
2000-01-03,91.617035,NaN
2000-01-04,88.034309,-0.039891
2000-01-05,88.191727,0.001787
2000-01-06,86.774353,-0.016202
2000-01-07,91.813919,0.056453


# Stationary Block Boostrapping (SBB)

In order to realistically simulate the SPY returns, there are two important properties that must hold: (i) Convergence in Distribution and (ii) Volatility Clustering.

Therefore, we will utilize the SBB due to the following properties:
- **Local Stationarity Holds Over Short Horizons**: The bootstrap resamples short, contiguous blocks in which the distribution of returns is approximately stable. By stitching these blocks together, it preserves short-range dependence.
- **Convergence in Distribution**: As $n \to \infty$ and $t \to \infty$ at a slower rate than $n$, the bootstrap distribution converges to the same asymptotic distribution as the original data.
- **Volatility Clustering**: Financial returns exhibit autocorrelation and breaking structure also destroys this property. However, block bootstrap preserves short-range dependence within blocks and will be at least partially preserved.

Nonetheless, there will be limitations:
- **Finite-sample issue**: With short samples or poorly chosen block length, volatility dynamics may be distorted.
- **Long-memory volatility**: Requires very large blocks, which reduces resample diversity.
- **No new volatility shocks or unseen data**: Bootstrap cannot create what was not observed.

In [ ]:
def stationary_bootstrap(returns:np.ndarray, n_sims:int=1, t:int=252, avg_block_size:int=None, seed:int=None) -> np.ndarray:
    rng = np.random.default_rng(seed)
    T = len(returns)
    sims = np.zeros((n_sims, t))
    if avg_block_size is None:
        avg_block_size = int(np.floor(T**(1/3)))
    
    restart_prob = 1/avg_block_size
    
    for i in range(n_sims):
        array_idx = 0
        while array_idx < t:
            if array_idx == 0 or rng.random() < restart_prob:
                idx = rng.integers(T)
            else:
                idx = (idx + 1) % T #Implement circular indexing
            
            sims[i, array_idx] = returns[idx]
            array_idx += 1
    
    return sims

In [15]:
initial_params = {
    'returns': df['log_return'].dropna().to_numpy(),
    'n_sims': 1,
    't': 5000,
    'avg_block_size': None,
    'seed': 123
}

initial_bootstrap = stationary_bootstrap(**initial_params)
initial_bootstrap

array([[ 0.00113289,  0.00060227,  0.0038868 , ..., -0.00419623,
         0.00760238, -0.0140082 ]], shape=(1, 5000))

# Automatic Block Size Selection Methodology
In block bootstrapping, increasing the block size reduces bias because larger blocks better preserve the original, long-range temporal dependence structure of the data. however, excessively large blocks reduce the number of independent resampling blocks, leading to higher variance in the bootstrap estimator. Consequently, block length selection involves a bias–variance tradeoff, and the optimal block size is chosen to be the smallest value that adequately captures the dependence structure while minimizing variance.

With that in mind and the fact that time-series data is typically dependent on previous values, we have the following notation for long-run variance:
$$\sigma^2_\infty = \gamma(0) + 2\sum^\infty_{k=1}\gamma(k)$$
where $\gamma(k) = Cov[X_0, X_k]$ and where $k$ is the lag.

## Adaptive Bandwidth Choice
The best expected blocksize for the SBB is determined by minimizing the MSE as denoted:
$$
\text{MSE}(\hat{\sigma}_b^2) = 
\underbrace{\text{Bias}^2}_{\text{block too short}} + 
\underbrace{\text{Variance}}_{\text{blocks too long}}
$$

Here, we can break it down into its components:
$$Bias[\hat{\sigma}^2_b] = E[\sigma^2_b] - \sigma^2_{\infty} = -\frac{1}{b}G + o\left(\frac{1}{b}\right)$$
$$V[\hat{\sigma}^2_b] = \frac{b}{N}D + o\left(\frac{b}{N}\right)$$
$$G = \sum^\infty_{k=-\infty}|k|R(k) \qquad D = 2g^2(0) \qquad g(\omega) = \sum^\infty_{s=-\infty}R(s)cos(\omega s)$$

The above equations have some confusing notations, and hence we will clarify the intent and meaning of each:
- $o\left(\frac{1}{b}\right)$ and $o\left(\frac{b}{N}\right)$ indicate that the remainder terms disappear faster than the primary terms as the sample size $N$ and block size $b$ grows.
- $\sigma^2_{\infty}$ is the true asymptotic variance of the sample mean.
- $D$ represents the variance constant for the SBB or how noisy each block-level contribution is.
- $g(\omega)$ is the spectral density function, if your process has large low-frequency power (strong persistence / volatility clustering), every block carries a lot of correlated noise.
- $R(k)$ or $R(s)$ is simply a different notation for $\gamma(k)$ and $\gamma(s)$, they have equivalent meaning.

Lastly, $G$ is harder to understand by itself. It is easier to understand if we look at how it affects the Bias term.
$$Bias[\hat{\sigma}^2_b]= -\frac{1}{b}G + o\left(\frac{1}{b}\right)$$
 
The negative sign is included because the SBB cuts the time series into chunks, destroying some correlation and that means less long-run variance, leading to an estimate that is bias downwards. Moreover, at lag $k$, dependence survive with probability $1 - \frac{|k|}{b}$ and broken with probability $\frac{|k|}{b}$. Naturally, this leads to the following weighted-sum equation:
$$Bias[\hat{\sigma}^2_b] \approx -\sum^\infty_{k=-\infty}\frac{|k|}{b}R(k)$$
$$Bias[\hat{\sigma}^2_b] \approx -\frac{1}{b}G$$

This leads us to the final MSE equation:
$$\text{MSE}(\hat{\sigma}_b^2) = \frac{G^2}{b^2} + D\frac{b}{N} + o(b^{-2}) + o(b/N)$$

If we want to minimize MSE, we would choose:
$$b = \left(\frac{2G^2}{D}\right)^{1/3} N^{1/3}$$
$$\text{MSE}(\hat{\sigma}_b^2) \approx \frac{3}{2^{2/3}}\frac{G^{2/3}D^{2/3}}{N^{2/3}}$$

## Flat-Top Lag-Window